In [42]:
import os.path 
import pandas as pd
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv

from pandas import datetime
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy

D:\Users\aumar\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  import sys


In [ ]:
directory =os.path.dirname(os.getcwd())

In [2]:
# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

# create a differenced series to make them stationary
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [7]:
# load dataset
shampoo=pd.read_csv(directory+'\\Data\\shampo_sales.csv',header=0,names=['Sales'])
# transform data to be stationary
raw_shampoo = shampoo.values
diff_shampoo = difference(raw_shampoo, 1)
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_shampoo, 4)
supervised_values = supervised.values

# split data into train and test-sets
train_data, test_data = supervised_values[0:-12,:], supervised_values[-12:,:]
Train_X=train_data[:, 0:-1]
Train_X = Train_X.reshape(Train_X.shape[0], 1, Train_X.shape[1])
Train_Y=train_data[:, -1]
Test_X=test_data[:, 0:-1]
Test_X = Test_X.reshape(Test_X.shape[0], 1, Test_X.shape[1])
Test_Y=test_data[:, -1]

# Model definition

In [8]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, Train_X.shape[1], Train_X.shape[2]), stateful=True))
model.add(Dense(1))


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
EPOCHS = 3000

In [12]:
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(EPOCHS):
    model.fit(Train_X, Train_Y, epochs=1, batch_size=1, verbose=0, shuffle=False)
    model.reset_states()


# Test Analysis

In [13]:
Train_X.shape

(23, 1, 4)

In [15]:
# forecast the entire training dataset to build up state for forecasting
model.predict(Train_X, batch_size=1)

array([[-109.21513  ],
       [  32.81757  ],
       [ -61.849033 ],
       [  11.891741 ],
       [  42.879803 ],
       [  -2.4843707],
       [  -6.0621796],
       [ -33.33722  ],
       [  23.718788 ],
       [ 118.89114  ],
       [ -33.474358 ],
       [  25.629417 ],
       [  32.81757  ],
       [  73.25985  ],
       [  67.72335  ],
       [ -83.01368  ],
       [  23.718788 ],
       [ -65.43468  ],
       [  47.47849  ],
       [ -12.14569  ],
       [  66.07163  ],
       [ -40.660767 ],
       [  -2.4845695]], dtype=float32)

In [21]:
X.shape

(1, 4)

In [26]:
# walk-forward validation on the test data
predictions = list()
for i in range(len(test_data)):
    # make one-step forecast
    X, y = Test_X[i,:],Test_Y[i]
    X= X.reshape(1, 1, 4)
    yhat = model.predict(X, batch_size=1)
    # invert scaling
    #yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_shampoo, yhat, len(test_data)+1-i)
    # store forecast
    predictions.append(yhat)
    expected = raw_shampoo[len(train_data) + i + 1]
    print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))


Month=1, Predicted=425.233868, Expected=339.700000
Month=2, Predicted=390.315284, Expected=440.400000
Month=3, Predicted=435.182233, Expected=315.900000
Month=4, Predicted=313.415430, Expected=439.300000
Month=5, Predicted=405.758130, Expected=401.300000
Month=6, Predicted=398.812394, Expected=437.400000
Month=7, Predicted=463.029417, Expected=575.500000
Month=8, Predicted=542.027946, Expected=407.600000
Month=9, Predicted=440.417570, Expected=682.000000
Month=10, Predicted=608.086113, Expected=475.300000
Month=11, Predicted=515.326447, Expected=581.300000
Month=12, Predicted=614.117570, Expected=646.900000


In [39]:
predictions=numpy.reshape(numpy.array(predictions),(12,1))


In [37]:
raw_shampoo[-12:].shape

(12, 1)

In [43]:
rmse = sqrt(mean_squared_error(raw_shampoo[-12:], predictions))
print('Test RMSE: %.3f' % rmse)
rsq = r2_score(raw_shampoo[-12:], predictions) #R-Squared on the testing data
print('R-square, Testing: ',rsq)

Test RMSE: 113.119
R-square, Testing:  -0.016041763053766145
